In [60]:
# using the 39 phone set proposed in (Lee & Hon, 1989)
# Table 3. Mapping from 61 classes to 39 classes, as proposed by Lee and Hon, (Lee & Hon,
# 1989). The phones in the left column are folded into the labels of the right column. The
# remaining phones are left intact.}
!pip install trax
pip install --upgrade jax jaxlib
pip install --upgrade trax
import trax
import trax.layers as tl
import random as rnd
import tensorflow as tf
import json
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

import itertools
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array



from trax.fastmath import numpy as fastnp
import numpy as np
import os


In [2]:

def collect_video_files(data_type, resp, root_dir):
    if resp == 'lipspeakers':
        speakerDir = root_dir+'/lipspeakers'
    elif resp == 'volunteers':
        speakerDir = root_dir+'/volunteers'
    else:
        speakerDir = root_dir

    fileName = f'{data_type}_dict.json'
    data = open(speakerDir+os.sep+fileName).read()
    data_dict = json.loads(data)

    with open(f'{speakerDir}/data_phn.txt', 'r') as f:
        lines = f.readlines()
    video_list = [l.strip() for l in lines]

    return data_dict, video_list, speakerDir




video_path = os.path.join('/kaggle/input/volunt')
spkr_type = 'volunteerss'

lip_data_dict, lip_video_list, lip_data_dir = collect_video_files('data', 'volunteers',video_path)


rnd.shuffle(lip_video_list)
lip_video_list=list(set(lip_video_list))
train_size=int(len(lip_video_list)*.9)
train_list=lip_video_list[:train_size]

val_list=lip_video_list[train_size:]

In [3]:
len(train_list),len(val_list)

(4672, 520)

In [4]:
phoneme_set_39_list = [
    'iy', 'ih', 'eh', 'ae', 'ah', 'uw', 'uh', 'aa', 'ey', 'ay', 'oy', 'aw', 'ow',
    'l', 'r', 'y', 'w', 'er', 'm', 'n', 'ng', 'ch', 'jh', 'dh', 'b', 'd', 'dx',
    'g', 'p', 't', 'k', 'z', 'v', 'f', 'th', 's', 'sh', 'hh', 'sil'
]
def get_vocabs(phoneme_set_39_list):
    phonem_nunmber={'<pad>':0}
    number_phonem={0:'<pad>'}
    for phonem in phoneme_set_39_list:
        phonem_nunmber[phonem]=len(phonem_nunmber)
        number_phonem[len(number_phonem)]=phonem
    return phonem_nunmber,number_phonem

phonem_nunmber,number_phonem=get_vocabs(phoneme_set_39_list)
len(phonem_nunmber),len(number_phonem)

(40, 40)

In [5]:
mobile = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, 
                                                weights='imagenet',input_shape=(224,224,3))
output=mobile.output
output=Flatten(name='flatten')(output)
output=Dense(1024,activation='relu')(output)
model = Model(inputs=mobile.input, outputs=output)


9406464/9406464 [==============================] - 0s 0us/step


In [77]:
def generte_date(data,data_labels,data_dir,phonem_nunmber,model):
    data_size=len(data)
    idx=0
    while True:
        if idx>=data_size:
            idx=0


        video=data[idx]
        spk,vdo = video.split('_')[:2]
        image_name = vdo.split('.')[0]+'_'

        list_IDs_temp = sorted([image_file for image_file in list(data_labels) if image_name in image_file])
        list_IDs_temp=sorted([image_file for image_file in list_IDs_temp if image_file in os.listdir(data_dir+'/'+spk)])

        idx+=1
        X = np.zeros((len(list_IDs_temp),224,224,3))
        Y = []
        for i, ID in enumerate(list_IDs_temp):

            image = load_img(data_dir+'/'+spk+'/'+ID,target_size=(224,224))
            image=img_to_array(image)
            image=preprocess_input(image)
            X[i] = image

            Y.append(phonem_nunmber[data_labels[ID]])
        X=model.predict(X,verbose=0)
        Y=np.array(Y)
        yield ((X,Y))

In [78]:
X,Y=next(generte_date(train_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))
X.shape,Y.shape

((34, 1024), (34,))

In [79]:
boundaries =  [8,   16,  32, 64, 128, 256]
batch_sizes = [128, 64, 32, 16,    8,   4,2]
# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(generte_date(train_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))

val_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(generte_date(val_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))


train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
val_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(val_batch_stream)



In [82]:
%%time
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

CPU times: user 2min 11s, sys: 3min 28s, total: 5min 39s
Wall time: 30.4 s


((16, 64, 1024), (16, 64), (16, 64))

In [86]:
# UNQ_C1
# GRADED FUNCTION
def input_encoder_fn(n_encoder_layers):
    """ Input encoder runs on the input sentence and creates
    activations that will be the keys and values for attention.
    
    Args:
        input_vocab_size: int: vocab size of the input
        d_model: int:  depth of embedding (n_units in the LSTM cell)
        n_encoder_layers: int: number of LSTM layers in the encoder
    Returns:
        tl.Serial: The input encoder
    """
    
    # create a serial network
    input_encoder = tl.Serial( 

        # feed the embeddings to the LSTM layers. It is a stack of n_encoder_layers LSTM layers
        [tl.LSTM(n_units=1024) for _ in range(n_encoder_layers)]
        ### END CODE HERE ###
    )

    return input_encoder

In [87]:
# UNQ_C2
# GRADED FUNCTION
def pre_attention_decoder_fn(mode, target_vocab_size, d_model):
    """ Pre-attention decoder runs on the targets and creates
    activations that are used as queries in attention.
    
    Args:
        mode: str: 'train' or 'eval'
        target_vocab_size: int: vocab size of the target
        d_model: int:  depth of embedding (n_units in the LSTM cell)
    Returns:
        tl.Serial: The pre-attention decoder
    """
    
    # create a serial network
    pre_attention_decoder = tl.Serial(
        
        ### START CODE HERE ###
        # shift right to insert start-of-sentence token and implement
        # teacher forcing during training
        tl.ShiftRight(),

        # run an embedding layer to convert tokens to vectors
        tl.Embedding(target_vocab_size,d_model),

        # feed to an LSTM layer
        tl.LSTM(d_model)
        ### END CODE HERE ###
    )
    
    return pre_attention_decoder

In [88]:
# UNQ_C3
# GRADED FUNCTION
def prepare_attention_input(encoder_activations, decoder_activations, inputs):
    """Prepare queries, keys, values and mask for attention.
    
    Args:
        encoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the input encoder
        decoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the pre-attention decoder
        inputs fastnp.array(batch_size, padded_input_length): input tokens
    
    Returns:
        queries, keys, values and mask for attention.
    """
    
    ### START CODE HERE ###
    print(f'encoder_activations is {encoder_activations.shape}')
    # set the keys and values to the encoder activations
    keys = encoder_activations
    values = encoder_activations

    
    # set the queries to the decoder activations
    queries = decoder_activations
    
    # generate the mask to distinguish real tokens from padding
    # hint: inputs is positive for real tokens and 0 where they are padding
    mask =np.sum(inputs,axis=2)
    mask=mask!=0
    ### END CODE HERE ###
    
    # add axes to the mask for attention heads and decoder length.
    mask = fastnp.reshape(mask, (mask.shape[0], 1, 1, mask.shape[1]))
    
    # broadcast so mask shape is [batch size, attention heads, decoder-len, encoder-len].
    # note: for this assignment, attention heads is set to 1.
    mask = mask + fastnp.zeros((1, 1, decoder_activations.shape[1], 1))
    
        
    
    return queries, keys, values, mask

In [90]:
# UNQ_C4
# GRADED FUNCTION
def NMTAttn(input_vocab_size=33300,
            target_vocab_size=len(phonem_nunmber),
            d_model=1024,
            n_encoder_layers=2,
            n_decoder_layers=2,
            n_attention_heads=1,
            attention_dropout=0.1,
            mode='train'):
    """Returns an LSTM sequence-to-sequence model with attention.

    The input to the model is a pair (input tokens, target tokens), e.g.,
    an English sentence (tokenized) and its translation into German (tokenized).

    Args:
    input_vocab_size: int: vocab size of the input
    target_vocab_size: int: vocab size of the target
    d_model: int:  depth of embedding (n_units in the LSTM cell)
    n_encoder_layers: int: number of LSTM layers in the encoder
    n_decoder_layers: int: number of LSTM layers in the decoder after attention
    n_attention_heads: int: number of attention heads
    attention_dropout: float, dropout for the attention layer
    mode: str: 'train', 'eval' or 'predict', predict mode is for fast inference

    Returns:
    An LSTM sequence-to-sequence model with attention.
    """

    ### START CODE HERE ###
    
    # Step 0: call the helper function to create layers for the input encoder
    input_encoder = input_encoder_fn(n_encoder_layers)

    # Step 0: call the helper function to create layers for the pre-attention decoder
    pre_attention_decoder = pre_attention_decoder_fn(mode, target_vocab_size, d_model)

    # Step 1: create a serial network
    model = tl.Serial( 
        
      # Step 2: copy input tokens and target tokens as they will be needed later.
      tl.Select([0,1,0,1]),
        
      # Step 3: run input encoder on the input and pre-attention decoder the target.
      tl.Parallel(input_encoder,pre_attention_decoder),
        
      # Step 4: prepare queries, keys, values and mask for attention.
      tl.Fn('PrepareAttentionInput', prepare_attention_input, n_out=4),
        
      # Step 5: run the AttentionQKV layer
      # nest it inside a Residual layer to add to the pre-attention decoder activations(i.e. queries)
      tl.Residual(tl.AttentionQKV(d_model, n_heads=n_attention_heads, dropout=attention_dropout, mode=mode)),
      
      # Step 6: drop attention mask (i.e. index = None
      tl.Select([0,2]),
        
      # Step 7: run the rest of_de the RNN decoder
      [tl.LSTM(d_model) for _ in range(n_decoder_layers)],
        
      # Step 8: prepare output by making it the right size
      tl.Dense(target_vocab_size),   
      # Step 9: Log-softmax for output
      tl.LogSoftmax()
    )
    
    ### END CODE HERE
    
    return model

In [91]:
# UNQ_C5
# GRADED PART
from trax.supervised import training
def train_task_function(train_batch_stream):
    """Returns a trax.training.TrainTask object.

    Args:
    train_batch_stream generator: labeled data generator

    Returns:
    A trax.training.TrainTask object.
    """
    return training.TrainTask(

        ### START CODE HERE

        # use the train batch stream as labeled (
        labeled_data= train_batch_stream,

        # use the cross entropy loss
        loss_layer= tl.CrossEntropyLoss(),

        # use the Adam optimizer with learning rate of 0.01
        optimizer= trax.optimizers.Adam(0.001),

        # use the `trax.lr.warmup_and_rsqrt_decay` as th.01e learning rate schedule
        # have 1000 warmup steps with a max value of 0.01
        

        # have a checkpoint every 10 steps
        n_steps_per_checkpoint= 10

        ### END CODE HERE
    )

In [92]:
train_task = train_task_function(train_batch_stream)

ImportError: cannot import name 'HashablePartial' from 'jax._src.util' (/usr/local/lib/python3.8/site-packages/jax/_src/util.py)
  In call to configurable 'Adam' (<class 'trax.optimizers.adam.Adam'>)

In [16]:

eval_task = training.EvalTask(
    
    ## use the eval batch stream as labeled data
    labeled_data=val_batch_stream,
    
    ## use the cross entropy loss and accuracy as metrics
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)


In [22]:
# define the output directory
output_dir = 'output_dir/'


# define the training loop
training_loop = training.Loop(NMTAttn(mode='train'),
                              train_task,
                              eval_tasks=[eval_task],
                              output_dir=output_dir)

2023-05-30 17:26:59.310080: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2432] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 83886464 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:        16B
              constant allocation:         0B
        maybe_live_out allocation:   32.00MiB
     preallocated temp allocation:   80.00MiB
  preallocated temp fragmentation:         0B (0.00%)
                 total allocation:  112.00MiB
              total fragmentation:   16.00MiB (14.29%)
Peak buffers:
	Buffer 1:
		Size: 32.00MiB
		Operator: op_name="jit(_uniform)/jit(main)/max" source_file="/tmp/ipykernel_29/2642057364.py" source_line=6
		XLA Label: fusion
		Shape: f32[2048,4096]

	Buffer 2:
		Size: 16.00MiB
		Operator: op_name="jit(_uniform)/jit(main)/threefry2x32" source_file="/tmp/ipykernel_29/2642057364.py" source_line=6
		XLA Label: custom-call
		Shape: u32[4194304]

	Buffer 3:
		Size:

LayerError: Exception passing through layer Serial (in init):
  layer created in file [...]/tmp/ipykernel_29/3503318463.py, line 39
  layer input shapes: (ShapeDtype{shape:(16, 64, 1024), dtype:float64}, ShapeDtype{shape:(16, 64), dtype:int64}, ShapeDtype{shape:(16, 64), dtype:float32})

  File [...]/trax/layers/combinators.py, line 107, in init_weights_and_state
    sublayer.init(inputs, use_cache=True))

LayerError: Exception passing through layer Parallel (in init):
  layer created in file [...]/tmp/ipykernel_29/3503318463.py, line 45
  layer input shapes: (ShapeDtype{shape:(16, 64, 1024), dtype:float32}, ShapeDtype{shape:(16, 64), dtype:int32})

  File [...]/trax/layers/combinators.py, line 226, in init_weights_and_state
    inits = [layer.init(signature, use_cache=True)

  File [...]/trax/layers/combinators.py, line 226, in <listcomp>
    inits = [layer.init(signature, use_cache=True)

LayerError: Exception passing through layer Serial (in init):
  layer created in file [...]/tmp/

In [ ]:
n_step=int(len(train_list)/32)*100
print(n_step)
training_loop.run(n_step)


146
37
25
30
34
31
33
47
39
33
35
58
58
54
52
17
33
31
50
31
26
37
56
40
35
33
57
42
30
44
43
28
61
29
39
33
31
36
29
54
20
36
16
30
22
43
34
43
32
34
encoder_activations is (4, 64, 1024)
shape of mask is (4, 1, 64, 64)

Step      1: Total number of trainable weights: 247668776
Step      1: Ran 1 train steps in 442.16 secs
Step      1: train CrossEntropyLoss |  4.13491631
29
45
43
32
32
47
35
17
31
36
47
18
30
27
41
38
24
31
45
41
57
30
25
57
41
23
34
36
24
38
28
34
29
33
40
49
40
39
31
36
39
25
30
29
40
39
22
46
34
54
44
41
37
encoder_activations is (4, 64, 1024)
shape of mask is (4, 1, 64, 64)
Step      1: eval  CrossEntropyLoss |  3.69793653
Step      1: eval          Accuracy |  0.02941176
34
23
46
39
36
40
14
30
41
28
13
32
18
31
29
12
16
21
27
27
45
33
30
32
42
31
61
17
23
45
25
57
52
31
28
20
38
42
58
38
31
45
33
40
31
31
58
30
29
33
32
20
42
50
29
19
37
50
28
encoder_activations is (8, 32, 1024)
shape of mask is (8, 1, 32, 32)

Step      2: Ran 1 train steps in 464.60 secs
Step

In [33]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working/output_dir'
remove_folder_contents(folder_path)
os.rmdir(r'/kaggle/working/output_dir')

In [85]:
#(batch,timestep)
#embedding
#(batch,timestep,n_dimension)
#(batch,timestep,w,h,channel)
#(batch,timestep,1024)